# Generalized Multiline Queues

In [1]:
import itertools 
import copy
from sage.combinat.q_analogues import *
from sage.combinat.sf.sf import *
from sage.combinat.ncsf_qsym.qsym import *
from sage.rings.rational_field import *
from sage.combinat.subset import *
from sage.combinat.permutation import *
from sage.misc.latex import *

import sage.combinat.permutation as permutation
from sage.combinat.sf.macdonald import qt_kostka
from sage.combinat.sf.ns_macdonald import E
from sage.rings.polynomial.polydict import ETuple

coeffs_ring = ZZ['q','t']

q = coeffs_ring.gens()[0]
t = coeffs_ring.gens()[1]

coeffs_field = coeffs_ring.fraction_field()
R=PolynomialRing(coeffs_field,11,'x')
xs=list(R.gens())

sym = SymmetricFunctions(coeffs_field)
h = sym.homogeneous()
m = sym.monomial()
s = sym.schur()
e = sym.elementary()
p = sym.power()
Ht = sym.macdonald().Ht()
H = sym.macdonald().H()
MP = sym.macdonald().P()
W = sym.macdonald(t=0).P()
MJ = sym.macdonald().J()
HLP = sym.hall_littlewood(t).P()
HLQ = sym.hall_littlewood(t).Q();
HLQp = sym.hall_littlewood(t).Qp();
JJ = sym.jack().J()
qsym = QuasiSymmetricFunctions(coeffs_field)
F = qsym.Fundamental()
M = qsym.Monomial()
QS = qsym.QS()
YQS = qsym.YQS()

## Object Construction

In [2]:
# COORDINATES START AT 1
# Coordinates are given in the usual way (i,j) means i to the right, j up

def balls_coordinates(balls):
    coords = []
    w = len(balls)
    l = len(balls[0])
    for i in range(w):
        for j in range(l):
            if balls[i][j] == 1: 
                coords.append([i+1,j+1])
            elif balls[i][j] == 2:
                coords.append([i+1,j+1,"b"])
            elif balls[i][j] == -2:
                coords.append([i+1,j+1,"r"]) 
    return(coords)
    

# find the queue index in which elem is
def find_queue(self,elem):
    ind = -1
    for i in range(len(self.queues)):
        queue = self.queues[i]
        if elem in queue:
            ind = i
            break
    if ind == -1:
        return("element not queued")
    else:
        return(ind)  
    
# word is a ternary word with 0,1,2 and 1=(, 2=) and 0=blank, to perform parenthesis matching algorithm
# returns the leftover letters from row1 after matching all possible 1s and 2s, and deleting unmatched 2s
def match_parenthesis(w):
    word = w.copy()
    boo = True
    while boo:
        p1 = -1
        p2 = -1
        for i in range(len(word)):
            if word[i] == 1:
                p1 = i
            elif word[i] == 2:
                p2 = i
            if p2 != -1:
                word[p2] = 0
                if p1 != -1:
                    word[p1] = 0        
                break
        if p2==-1:
            boo = False
    return(word)
    
# collapse row 1 on top of row 2
# this method updates row1 and row2
# wit is a boolean variable that says if a collapse occurred or not
def two_row_collapse(row1,row2):
    par_word = []
    wit = False
    # Create the parenthesis word to do the matching algorithm
    for i in range(len(row1)):
        if row1[i]==1:
            par_word.append(1)
        else:
            par_word.append(0)
        if row2[i]==1:
            par_word.append(2)
        else:
            par_word.append(0)
    # Apply parenthesis matching
    red_par_word = match_parenthesis(par_word)
    # Find indices on row1 to collapse
    coll = [red_par_word[i] for i in range(len(red_par_word)) if i%2==0]    
    for i in range(len(row1)):
        if coll[i] == 1:
            wit = True
            row1[i] = 0
            row2[i] = 1
    return(row1.copy(),row2.copy(),wit)


# Print a list of lists in a nice way
def print_list_of_lists(L):
    #print("--------------------------")
    print('\n'.join(' '.join('%2d' % x for x in l) for l in L))
    #print("--------------------------")

    
##   GENERALIZED MULTILINE QUEUE CLASS   ##

class GeneralizedMultilineQueue:
    pairings = []
    case = 0
    r = 0
    set_case = False
    bool_labels = False
    bool_pairings = False
        
    # size = [width,height]
    
    def __init__(self,balls):
        self.balls = deepcopy(balls)
        #Matrix of 0s for labels
        self.labels = [[0 for i in range(len(balls[0]))] for j in range(len(balls))]   
        self.size = [len(balls[0]),len(balls)]
        self.QM =[ [] for _ in range(self.size[1]) ]
        
    # This method creates the queues
    # 
    # case=0:   pairing particles to the right, then antiparticles and remaining particles to the left. 
    #
    #
    # case=1:   AGS Pairing
    #           
    #
    # case=2:   
    #          
    #           
    # case=3:  
    #           
    #           
    # case=4:   
    #           
    #

    
    def pair(self,case):
        
        if not self.set_case:
            self.case = case
            self.set_case = True
        
        self.pairings = []
        
        if case==0:  
            
            #Update boolean variable
            self.bool_labels = True
            
            #Update labels of the first row
            for i in range(self.size[0]):
                #If ball is particle:
                if self.balls[-1][i] == 1:
                    self.labels[-1][i] = self.size[1]
                #If ball is antiparticle:
                if self.balls[-1][i] == 0:
                    self.labels[-1][i] = self.size[1]-1
                        
            for i in reversed(range(1,self.size[1])):
                
                self.bool_pairings = True
                
                top_row = self.balls[i].copy()
                bot_row = self.balls[i-1].copy()
                
                #Find particles in the top row
                part_top = [j for j in range(self.size[0]) if top_row[j] == 1]
                #Find antiparticles in the top row
                antipart_top = [j for j in range(self.size[0]) if top_row[j] == 0]
                
                #Ordered lists according to label
                ind_part_top = []
                ind_antipart_top = []
                
                #Sort particles with respect to their current label:
                
                #Order particles in decreasing label order
                for lab in reversed(range(self.size[1]+1)):
                    for p in part_top:
                        if self.labels[i][p] == lab and self.balls[i][p] == 1:
                            ind_part_top.append(p)
                
                #Order antiparticles in increasing label order
                for lab in range(self.size[1]+1):
                    for p in antipart_top:
                        if self.labels[i][p] == lab and self.balls[i][p] == 0:
                            ind_antipart_top.append(p)
                
                #Find particles in the bottom row
                ind_part_bot = [j for j in range(self.size[0]) if bot_row[j] == 1]
                #Find antiparticles in the bottom row
                ind_antipart_bot = [j for j in range(self.size[0]) if bot_row[j] == 0]
                               
                # 1) Pair particles from row i
                
                rem_particles_top = ind_part_top
                rem_particles_bot = ind_part_bot
                
                stop = False
                
                while len(ind_part_top) > 0 and len(ind_part_bot) > 0 and not stop:
                    
                    k = ind_part_top[0]

                    #Look for particle weakly to the right
                    poss_pairings_right = [l for l in range(k,self.size[0]) if l in ind_part_bot]
                    poss_pairings_left = [l for l in range(k) if l in ind_part_bot]
                    
                    
                    if len(poss_pairings_right) > 0:
                        #Pair particle k to particle min(possibility to the right)
                        p = min(poss_pairings_right)
                        
                        #Update the pairing list
                        self.pairings.append([i,k,p,"particle","right"])
                        #Update the labels list
                        self.labels[i-1][p] = deepcopy(self.labels[i][k])
                        
                        #Remove particles paired from lists
                        ind_part_top.remove(k)
                        ind_part_bot.remove(p)

                        
                    elif len(poss_pairings_right) == 0 and len(poss_pairings_left) > 0:
                        #Pair particle k to particle min(possibility on the left)
                        p = min(poss_pairings_left)
                        
                        #Update the pairing list
                        self.pairings.append([i,k,p,"particle","right"])
                        #Update the labels list
                        self.labels[i-1][p] = deepcopy(self.labels[i][k])
                        
                        #Remove particles paired from lists
                        ind_part_top.remove(k)
                        ind_part_bot.remove(p)
                        
                        
                    elif len(poss_pairings_right) == 0 and len(poss_pairings_left) == 0:

                        rem_particles_top = ind_part_top
                        rem_particles_bot = ind_part_bot
                        
                        stop = True
                    
                    #In case there are remaining particles in the bot row:
                    if len(ind_part_top) == 0:
                        rem_particles_bot = ind_part_bot
                        
                        
                    
                # Add remaining particles as "antiparticles" to be paired according to the rules                    
                antipart_top = ind_antipart_top 
                # Add particles as "antiparticles" in increasing label order
                for lab in range(self.size[1]+1):
                    for p in rem_particles_top:
                        if self.labels[i][p] == lab:
                            antipart_top.append(p)
                
                #Particles added in the bottom do not need special ordering
                antipart_bot = ind_antipart_bot + (rem_particles_bot)
                
                for k in antipart_top:
                    
                    poss_pairings_left = [l for l in range(k+1) if l in antipart_bot]
                    poss_pairings_right = [l for l in range(k+1,self.size[0]) if l in antipart_bot]
                    
                    if len(poss_pairings_left)>0:
                        #Pair to the closest available to the left
                        p = max(poss_pairings_left)
                        
                        #Update the pairing list
                        
                        if self.balls[i-1][p] == 1:
                            self.pairings.append([i,k,p,"particle","left"])
                            #Update the labels list
                            self.labels[i-1][p] = deepcopy(self.labels[i][k])
                            
                        elif self.balls[i-1][p] == 0:
                            self.pairings.append([i,k,p,"antiparticle","left"])
                            #Update the labels list
                            self.labels[i-1][p] = deepcopy(self.labels[i][k])-1
                            
                        antipart_bot.remove(p)
                    
                    else:
                        #Pair to the closest available to the left
                        p = max(poss_pairings_right)
                        
                        #Update the pairing list
                        
                        if self.balls[i-1][p] == 1:
                            self.pairings.append([i,k,p,"particle","left"])
                            #Update the labels list
                            self.labels[i-1][p] = deepcopy(self.labels[i][k])
                        elif self.balls[i-1][p] == 0:
                            self.pairings.append([i,k,p,"antiparticle","left"])
                            #Update the labels list
                            self.labels[i-1][p] = deepcopy(self.labels[i][k])-1
                            
                        antipart_bot.remove(p)   

        if case==1:  
            #Update boolean variable
            self.bool_labels = True
            
            #Update labels of the first row
            for i in range(self.size[0]):
                #If ball is particle:
                if self.balls[-1][i] == 1:
                    self.labels[-1][i] = self.size[1]
                #If ball is antiparticle:
                if self.balls[-1][i] == 0:
                    self.labels[-1][i] = self.size[1]-1
                        
            for i in reversed(range(1,self.size[1])):
                
                self.bool_pairings = True
                
                top_row = self.balls[i].copy()
                bot_row = self.balls[i-1].copy()
                
                #Find particles in the top row
                part_top = [j for j in range(self.size[0]) if top_row[j] == 1]
                #Find antiparticles in the top row
                antipart_top = [j for j in range(self.size[0]) if top_row[j] == 0]
                
                #Ordered lists according to label
                ind_part_top = []
                ind_antipart_top = []
                
                #Sort particles with respect to their current label:
                
                #Order particles in decreasing label order
                for lab in reversed(range(self.size[1]+1)):
                    for p in part_top:
                        if self.labels[i][p] == lab and self.balls[i][p] == 1:
                            ind_part_top.append(p)
                
                #Order antiparticles in increasing label order
                for lab in range(self.size[1]+1):
                    for p in antipart_top:
                        if self.labels[i][p] == lab and self.balls[i][p] == 0:
                            ind_antipart_top.append(p)
                
                #Find particles in the bottom row
                ind_part_bot = [j for j in range(self.size[0]) if bot_row[j] == 1]
                #Find antiparticles in the bottom row
                ind_antipart_bot = [j for j in range(self.size[0]) if bot_row[j] == 0]
                               
                # 1) Pair particles from row i
                
                rem_particles_top = ind_part_top
                rem_particles_bot = ind_part_bot
                
                stop = False
                
                while len(ind_part_top) > 0 and len(ind_part_bot) > 0 and not stop:
                    
                    #Pick particle with largest index and maximally to the left
                    k = ind_part_top[0]
                    

                    #Look for particle weakly to the right
                    poss_pairings_right = [l for l in range(k,self.size[0]) if l in ind_part_bot]
                    poss_pairings_left = [l for l in range(k) if l in ind_part_bot]
                    
                    
                    if len(poss_pairings_right) > 0:
                        #Pair particle k to particle min(possibility to the right)
                        p = min(poss_pairings_right)
                        
                        #Update the pairing list
                        self.pairings.append([i,k,p,"particle","right"])
                        #Update the labels list
                        self.labels[i-1][p] = deepcopy(self.labels[i][k])
                        
                        #Remove particles paired from lists
                        ind_part_top.remove(k)
                        ind_part_bot.remove(p)

                        
                    elif len(poss_pairings_right) == 0 and len(poss_pairings_left) > 0:
                        #Pair particle k to particle min(possibility on the left)
                        p = min(poss_pairings_left)
                        
                        #Update the pairing list
                        self.pairings.append([i,k,p,"particle","right"])
                        #Update the labels list
                        self.labels[i-1][p] = deepcopy(self.labels[i][k])
                        
                        #Remove particles paired from lists
                        ind_part_top.remove(k)
                        ind_part_bot.remove(p)
                        
                        
                    elif len(poss_pairings_right) == 0 and len(poss_pairings_left) == 0:

                        rem_particles_top = ind_part_top
                        rem_particles_bot = ind_part_bot
                        
                        stop = True
                    
                    #In case there are remaining particles in the bot row:
                    if len(ind_part_top) == 0:
                        rem_particles_bot = ind_part_bot
                
                rem_top = len(rem_particles_top)
                rem_bot = len(rem_particles_bot)
                
                
                #Pair the rem_bot of the antiparticles on top to the right to the particles remaining:
                
                if rem_bot > 0:
                    
                    for a in range(rem_bot):
                        m = ind_antipart_top[-1]
                        max_lab = self.labels[i][m]
                        k = min([j for j in ind_antipart_top if self.labels[i][j] == max_lab])                            
                        
                        #Look for particle weakly to the right
                        poss_pairings_right = [l for l in range(k,self.size[0]) if l in ind_part_bot]
                        poss_pairings_left = [l for l in range(k) if l in ind_part_bot]


                        if len(poss_pairings_right) > 0:
                            #Pair particle k to particle min(possibility to the right)
                            p = min(poss_pairings_right)

                            #Update the pairing list
                            self.pairings.append([i,k,p,"particle","right"])
                            #Update the labels list
                            self.labels[i-1][p] = deepcopy(self.labels[i][k])

                            #Remove particles paired from lists
                            ind_antipart_top.remove(k)
                            ind_part_bot.remove(p)


                        elif len(poss_pairings_right) == 0 and len(poss_pairings_left) > 0:
                            #Pair particle k to particle min(possibility on the left)
                            p = min(poss_pairings_left)

                            #Update the pairing list
                            self.pairings.append([i,k,p,"particle","right"])
                            #Update the labels list
                            self.labels[i-1][p] = deepcopy(self.labels[i][k])

                            #Remove particles paired from lists
                            ind_antipart_top.remove(k)
                            ind_part_bot.remove(p)
                                          
                # Add remaining particles as "antiparticles" to be paired according to the rules                    
                antipart_top = ind_antipart_top 
                # Add particles as "antiparticles" in increasing label order
                for lab in range(self.size[1]+1):
                    for p in rem_particles_top:
                        if self.labels[i][p] == lab:
                            antipart_top.append(p)
                
                #Particles added in the bottom do not need special ordering
                antipart_bot = ind_antipart_bot + (rem_particles_bot)
                
                for k in antipart_top:
                    
                    poss_pairings_left = [l for l in range(k+1) if l in antipart_bot]
                    poss_pairings_right = [l for l in range(k+1,self.size[0]) if l in antipart_bot]         
                    
                    if len(poss_pairings_left)>0:
                        #Pair to the closest available to the left
                        p = max(poss_pairings_left)
                        
                        #Update the pairing list
                        
                        if self.balls[i-1][p] == 1:
                            self.pairings.append([i,k,p,"particle","left"])
                            #Update the labels list
                            self.labels[i-1][p] = deepcopy(self.labels[i][k])
                            
                        elif self.balls[i-1][p] == 0:
                            self.pairings.append([i,k,p,"antiparticle","left"])
                            #Update the labels list
                            self.labels[i-1][p] = deepcopy(self.labels[i][k])-1
                            
                        antipart_bot.remove(p)
                    
                    else:
                        #Pair to the closest available to the left
                        p = max(poss_pairings_right)
                        
                        #Update the pairing list
                        
                        if self.balls[i-1][p] == 1:
                            self.pairings.append([i,k,p,"particle","left"])
                            #Update the labels list
                            self.labels[i-1][p] = deepcopy(self.labels[i][k])
                        elif self.balls[i-1][p] == 0:
                            self.pairings.append([i,k,p,"antiparticle","left"])
                            #Update the labels list
                            self.labels[i-1][p] = deepcopy(self.labels[i][k])-1
                            
                        antipart_bot.remove(p)   
                        
                        
    # Updates the labels of the particles
    
    def update_labels(self):
        #Update boolean variable
        self.bool_labels = True
        
        for i in range(self.size[0]):
            #If ball is particle:
            if self.balls[-1][i] == 1:
                self.labels[-1][i] = self.size[1]
            #If ball is antiparticle:
            if self.balls[-1][i] == 0:
                self.labels[-1][i] = self.size[1]-1

        for i in reversed(range(0,self.size[1]-1)):
            #Look for pairings of the current row
            current_row_pairings = [pair for pair in self.pairings if pair[0] == i+1]
            
            for j in range(self.size[0]):
                #Look for the pairing that has j in the bottom
                p = [pair for pair in current_row_pairings if pair[2] == j][0]

                if p[3] == "particle":
                    lab = copy.deepcopy(self.labels[i+1][p[1]])
                    self.labels[i][j] = lab
                
                if p[3] == "antiparticle":
                    lab = copy.deepcopy(self.labels[i+1][p[1]])
                    self.labels[i][j] = lab-1
            
    # Draw a queue between pi and pf in a drawing win
    # t = type of pairing: either particle or antiparticle
    # d = direction of pairing
    # offsets = aestheticall stuff

    def draw_pairing(self,win,pi,pf,t,d,offset_x,offset_y):
        
        xi = pi[1]+0.5
        yi = pi[0]+0.5
        xf = pf[1]+0.5
        yf = pf[0]+0.5
        
        r = self.r
        #Pairing goes to the right:
        if d == "right":
            if xi <= xf:
                win += line([(xi+offset_x,yi-self.r),(xi+offset_x,yi-self.r-offset_y)],color=Color('blue'))
                win += line([(xi+offset_x,yi-self.r-offset_y),(xf+offset_x,yi-self.r-offset_y)],color=Color('blue'))
                win += line([(xf+offset_x,yi-self.r-offset_y),(xf+offset_x,yf+self.r)],color=Color('blue'))
            else:
                win += line([(xi+offset_x,yi-self.r),(xi+offset_x,yi-self.r-offset_y)],color=Color('blue'))
                win += line([(xi+offset_x,yi-self.r-offset_y),(self.size[0],yi-self.r-offset_y)],color=Color('blue'))
                win += line([(0,yi-self.r-offset_y),(xf+offset_x,yi-self.r-offset_y)],color=Color('blue'))
                win += line([(xf+offset_x,yi-self.r-offset_y),(xf+offset_x,yf+self.r)],color=Color('blue'))
            return(win)
    
        elif d=="left":
            
            if t == "particle":
                if xi < xf:                
                    win += line([(xi-offset_x,yi-self.r),(xi-offset_x,yi-self.r-offset_y)],color=Color('blue'))
                    win += line([(xi-offset_x,yi-self.r-offset_y),(0,yi-self.r-offset_y)],color=Color('blue'))
                    win += line([(self.size[0],yi-self.r-offset_y),(xf-offset_x,yi-self.r-offset_y)],color=Color('blue'))
                    win += line([(xf-offset_x,yi-self.r-offset_y),(xf-offset_x,yf+self.r)],color=Color('blue'))

                else:               
                    win += line([(xi-offset_x,yi-self.r),(xi-offset_x,yi-self.r-offset_y)],color=Color('blue'))
                    win += line([(xi-offset_x,yi-self.r-offset_y),(xf-offset_x,yi-self.r-offset_y)],color=Color('blue'))
                    win += line([(xf-offset_x,yi-self.r-offset_y),(xf-offset_x,yf+self.r)],color=Color('blue'))
                return(win)
            
            elif t == "antiparticle":
                if xi < xf:                
                    win += line([(xi-offset_x,yi-self.r),(xi-offset_x,yi-self.r-offset_y)],color=Color('red'))
                    win += line([(xi-offset_x,yi-self.r-offset_y),(0,yi-self.r-offset_y)],color=Color('red'))
                    win += line([(self.size[0],yi-self.r-offset_y),(xf-offset_x,yi-self.r-offset_y)],color=Color('red'))
                    win += line([(xf-offset_x,yi-self.r-offset_y),(xf-offset_x,yf+self.r)],color=Color('red'))

                else:               
                    win += line([(xi-offset_x,yi-self.r),(xi-offset_x,yi-self.r-offset_y)],color=Color('red'))
                    win += line([(xi-offset_x,yi-self.r-offset_y),(xf-offset_x,yi-self.r-offset_y)],color=Color('red'))
                    win += line([(xf-offset_x,yi-self.r-offset_y),(xf-offset_x,yf+self.r)],color=Color('red'))
                return(win)
            
                    
    # Gives the drawing of a MLQ that can be paired or not (in which case just shows the ball arrangement)             
    def draw(self,radius):
        win = Graphics()
        self.r = radius
        w = self.size[1]
        h = self.size[0]
        
        #Draw gray grid        
        for i in range(0,h+1,1):
            win += line([(i,0),(i,w)],color=Color('lightgray'))
        for i in range(0,w+1,1):    
            win += line([(0,i),(h,i)],color=Color('lightgray'))
        
        #Draw ball arrangement
        
        for i in range(self.size[1]):
            for j in range(self.size[0]):
                xb = j+1
                yb = i+1
                
                # If particle in coordinate (i,j) is a particle:
                if self.balls[i][j] == 1:
                    win+= circle((xb-0.5,yb-0.5), self.r, fill = False, color = Color('blue'))
                    if self.bool_labels:
                        lab = str(self.labels[i][j])
                        fs = 20-(1.0/4.0)*max([self.size[0],self.size[1]])
                        win+= text(lab,(xb-0.505,yb-0.525),color = Color('blue'),fontsize = fs,rotation = 0)

                
                
                # If antiparticle in coordinate (i,j) is a particle:
                if self.balls[i][j] == 0:
                    win += line([((xb-0.5)-(self.r),(yb-0.5)+(self.r)),
                                     ((xb-0.5)+(self.r),(yb-0.5)+(self.r))],color=Color('red'))
                        
                    win += line([((xb-0.5)+(self.r),(yb-0.5)+(self.r)),
                                     ((xb-0.5)+(self.r),(yb-0.5)-(self.r))],color=Color('red'))
                        
                    win += line([((xb-0.5)+(self.r),(yb-0.5)-(self.r)),
                                     ((xb-0.5)-(self.r),(yb-0.5)-(self.r))],color=Color('red'))
                        
                    win += line([((xb-0.5)-(self.r),(yb-0.5)-(self.r)),
                                     ((xb-0.5)-(self.r),(yb-0.5)+(self.r))],color=Color('red'))
                    if self.bool_labels:
                        lab = str(self.labels[i][j])
                        fs = 20-(1.0/4.0)*max([self.size[0],self.size[1]])
                        win+= text(lab,(xb-0.505,yb-0.525),color = Color('red'),fontsize = fs,rotation = 0)
                    
        #Draw pairings
    
        if self.bool_pairings:
            cont_r = 0
            cont_l = -1
            for pair in self.pairings:
                    
                    pi = [pair[0],pair[1]]
                    pf = [pair[0]-1,pair[2]]
                    t = pair[3]
                    d = pair[4]

                    N = self.size[1]
                    
                    
                    if d=="right":
                        cont_r += 1
                        offset_y = (1-2*self.r) - (0.0325)*(2*(cont_r%7+1)+1)
                        offset_x = 0.0175*(3-(cont_r%2))
                    elif d=="left":
                        cont_l += 1
                        cont_r += 1
                        offset_y = (1-2*self.r) - (0.0325)*(2*(cont_r%7+1)+1)
                        offset_x = 0.0175*(2-(cont_l%3-1))
                    
                    win = self.draw_pairing(win,pi,pf,t,d,offset_x,offset_y)
                
        win.axes(False)
        win.show()
        
        
    ##  ALGEBRAIC METHODS  ##
    
    def maj(self):
        maj = 0
        for pair in self.pairings:
            #Wrapping particle to the left
            if pair[-1] == "right" and pair[1] > pair[2]:
                
                maj += self.labels[pair[0]][pair[1]]-(pair[0]+1)+1
            
            #Wrapping particle to the left
            if pair[-1] == "left" and pair[1] < pair[2]:
                
                maj -= self.labels[pair[0]][pair[1]]-(pair[0]+1)+1
    
        return(maj)
    
    
    #returns the x-weight of the multiline queue. Does not need to be paired
    def xweight(self):
        R=PolynomialRing(coeffs_field,self.size[0],'x')
        xs = R.gens()
        wei = xs[0]**0
        for i in range(self.size[1]):
            for j in range(self.size[0]):
                if self.balls[i][j] == 1:
                    wei *= xs[j]            
        return(wei)
    
    #returns the x-weight of the multiline queue. Requires pairing
    # case: see pairing cases
    def qweight(self):
        wei = q**0
        #q statistic calculation depends on the pairing algorithm
        if self.case == 0 or self.case == 1:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    if cf < ci:
                        wei *= q**(l+1-queue[i-1][0]) 
                        
        elif self.case == 3 or self.case == 4:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    if cf > ci:
                        wei *= q**(l+1-queue[i-1][0])
        return(wei)
    
    #returns the x-weight of the multiline queue. Requires pairing
    # case: see pairing cases
    def q_coweight(self):
        wei = q**0
        #q statistic calculation depends on the pairing algorithm
        if self.case == 0 or self.case == 1:
            for queue in self.queues:
                cont = 0
                for i in reversed(range(len(queue)-1)):
                    ci = queue[i][1]
                    cf = queue[i+1][1]
                    if cf >= ci:
                        cont += 1
                
                    wei *= q**(cont)
                        
                        
        elif self.case == 3 or self.case == 4:
            for queue in self.queues:
                cont = 0
                for i in reversed(range(len(queue)-1)):
                    ci = queue[i][1]
                    cf = queue[i+1][1]
                    if cf <= ci:
                        cont += 1
                    wei *= q**(cont)
        return(wei)
    
    #returns the column word of the multiline queue: column number of the balls from left to right, and bottom to top
    def rw(self):
        w = []
        for i in range(self.size[1]):
            for j in (range(self.size[0])):
                if self.balls[i][j] == 1:
                    w.append(j+1)
        
        return(w)
    
    #returns the column word of the multiline queue: row number of the balls from top to bottom and left to right
    def cw(self):
        w = []
        for i in range(self.size[0]):
            for j in reversed(range(self.size[1])):
                if self.balls[j][i] == 1:
                    w.append(j+1)
        
        return(w)
    
    #returns the inverted column word of the multiline queue: row number of the balls from top to bottom and right to left
    def invcolw(self):
        w = []
        for i in reversed(range(self.size[0])):
            for j in reversed(range(self.size[1])):
                if self.balls[j][i] == 1:
                    w.append(j+1)
        
        return(w)
        

In [81]:
# Print a list of lists in a nice way
def print_list_of_lists(L):
    #print("--------------------------")
    print('\n'.join(' '.join('%2d' % x for x in l) for l in L))
    #print("--------------------------")

def GMLQ(balls,r,case):
    GM = GeneralizedMultilineQueue(balls)
    GM.pair(case)
    GM.draw(r)
    return(GM)

#  Returns [pos,indexes] where pos is either bot or top (position of remaining unmatched things) and 
#                              indexes are the positions of the unmatched particles 

def cyclic_parenthesis_matching(bot_row,top_row):
    pos = str("")
    N = len(top_row)
    
    ind_top = [j for j in range(N) if top_row[j] == 1]
    ind_bot = [j for j in range(N) if bot_row[j] == 1]
    
    top_to_pair = ind_top.copy()
    bot_to_pair = ind_bot.copy()

    #If there are more particles in the bottom row:
    if sum(top_row) < sum(bot_row):
        while len(top_to_pair) > 0 :
            for pos in top_to_pair:
                ind_right = [k for k in range(pos,N) if k in bot_to_pair]
                ind_left = [k for k in range(pos) if k in bot_to_pair]
                
                if len(ind_right) > 0:
                    n = min(ind_right)
                    if len([j for j in range(pos+1,n+1) if j in top_to_pair]) == 0:
                        top_to_pair.remove(pos)
                        bot_to_pair.remove(n)
                        break
                    
                elif len(ind_right) == 0 and len(ind_left)>0 :
                    n = min(ind_left)
                    if len([j for j in range(pos+1,N) if j in top_to_pair]) == 0 and len([j for j in range(0,n+1) if j in top_to_pair]) == 0:
                        top_to_pair.remove(pos)
                        bot_to_pair.remove(n)
                        break
        
        return(["bot",bot_to_pair])
    
    #If there are more particles in the top row:
    if sum(top_row) > sum(bot_row):
        aux_top = list(reversed(bot_row))
        aux_bot = list(reversed(top_row))
        ans = cyclic_parenthesis_matching(aux_bot,aux_top)
        
        return(["top",ans[1]])
    
    if sum(top_row) == sum(bot_row):
        return(["even",[]])

def sigma(balls,i):
    ans = deepcopy(balls)
    
    top_row = deepcopy(ans[i+1])
    bot_row = deepcopy(ans[i])
    
    N = len(top_row)
    
    cyclic = cyclic_parenthesis_matching(bot_row,top_row)
    
    if(cyclic[0] == "bot"):
        for j in cyclic[1]:
            ans[i][j] = 0
            ans[i+1][j] = 1 
        return(ans)
    
    if(cyclic[0] == "top"):
        for j in cyclic[1]:
            ans[i][N-j-1] = 1
            ans[i+1][N-j-1] = 0 
        return(ans)
    
    if(cyclic[0] == "even"):
        return(ans)
    
#This method updates the list!
def permute_rows(balls,p):
    w = p.reduced_word()
    current_balls = deepcopy(balls)
    for num in reversed(w):
        current_balls = sigma(current_balls,num-1)
    return(current_balls)
    

#Sorts a list of balls to have partition content, and gives the permutation that needs to be applied to do so
def ordered_balls(balls):
    row_content = [sum(balls[i]) for i in range(len(balls))]
    l = row_content
    L = [ (l[i],i+1) for i in range(len(l)) ]
    L.sort(reverse = True)
    sorted_l,per = zip(*L)
    p = Permutation(list(per)).inverse()
    sorted_balls = permute_rows(deepcopy(balls),p)
    return([sorted_balls,p])

## Algebraic Combinatorics

In [241]:
# Generating function \sum_G x^G q^maj(G) for L x n binary matrices

def charge_gen(L,n,case):
    R=PolynomialRing(coeffs_field,n,'x')
    xs = R.gens()

    aux_list = []

    for s in [list(sub) for sub in Subsets(range(n))]:
        v = [0 for x in range(n)]
        for j in s:
            v[j] = 1
        aux_list.append(v)

    binary_matrices = itertools.product(*[aux_list for i in range(L)])

    poly = 0
    
    for b in binary_matrices:
        GM = GeneralizedMultilineQueue(b)
        GM.pair(case)
        maj = GM.maj()
        
        poly += (GM.xweight())*q**maj
        
    return(poly)


# Generating function \sum_G x^G q^maj(G) for L x n binary matrices with row content alpha
# alpha : a composition with L parts

## This gives the q--Whittaker polynomial \lambda = \alpha^+ 

def charge_polynomial(n,alpha,case):
    #R=PolynomialRing(coeffs_field,n,'x')
    #xs = R.gens()

    aux_list = []

    for a in alpha:
        row = []
        subs = [list(sub) for sub in Subsets(range(n)) if len(sub) == a]
        for s in subs:
            v = [0 for x in range(n)]
            for j in s:
                v[j] = 1
            row.append(v)
        aux_list.append(row)

    matrices = itertools.product(*aux_list)

    poly = 0
    
    for binary in matrices:
        b = list(binary).copy()
        GM = GeneralizedMultilineQueue(b)
        GM.pair(case)
        maj = GM.maj() 
        poly += (GM.xweight())*q**maj
        
    return(poly)
    
    
# Gives the set of maj=0 GMLQs of size L x n.

def non_wrapping_GMLQs(L,n,case):
    aux_list = []
    r = 0.2
    
    non_wrap = []
    
    for s in [list(sub) for sub in Subsets(range(n))]:
        v = [0 for x in range(n)]
        for j in s:
            v[j] = 1
        aux_list.append(v)

    binary_matrices = itertools.product(*[aux_list for i in range(L)])
    
    for binary in binary_matrices:
        balls = list(binary).copy()
        GM = GeneralizedMultilineQueue(balls)
        GM.pair(case)
        if GM.maj() == 0:
            non_wrap.append(GM)
            
    return(non_wrap)


def schur_GMLQs(n,alpha,case):
    aux_list = []
    r=0.2

    for a in alpha:
        row = []
        subs = [list(sub) for sub in Subsets(range(n)) if len(sub) == a]
        for s in subs:
            v = [0 for x in range(n)]
            for j in s:
                v[j] = 1
            row.append(v)
        aux_list.append(row)

    matrices = itertools.product(*aux_list)
    
    poly = 0
    
    for binary in matrices:
        balls = list(binary).copy()
        GM = GeneralizedMultilineQueue(balls)
        GM.pair(case)
        if GM.maj() == 0:
            GM.draw(r)
            poly += GM.xweight()
            
    return(poly)
    

## Rotations and RSK